<a href="https://colab.research.google.com/github/SAWGraph/public/blob/main/UseCases/UC1-Testing/UC1-CQ2c/UC1_CQ2c_query_(7201).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The purpose of this notebook is to query the SAWGraph knowledge graph for NHD surface water flowlines that are at most 20 km downstream of the flowlines nearest landfill and Department of Defense (DoD) sites.

This is a demonstration of competency question 2c from Use Case 1: Testing.

- What surface water bodies are downstream at most 20 km from landfills or DoD sites?

# Setup

Here we set up SPARQLWrapper to work with our endpoint and create our query.

## Install & Import Statements

Install: The SPARQLWrapper libary provides tools for querying SPARQL endpoints. The sparql_dataframe library can be used with SPARQLWrapper to convert JSON results from a SPARQL query directly to a Pandas dataframe.

Import: See the inline comments for a brief rationale of each library.

In [1]:
%%capture
!pip install mapclassify --upgrade --quiet
!pip install SPARQLWrapper --upgrade --quiet
!pip install sparql_dataframe --upgrade --quiet

In [2]:
from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For maps
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper, JSON, GET, POST, DIGEST, get_sparql_dataframe   # For querying SPARQL endpoints
from tabulate import tabulate                                      # For pretty printing dataframes

## Variable Initialization

A SPARQLWrapper is created to access the Hydrology repository for the SAWGraph project.

In [3]:
%%capture
pd.options.display.width = 240

endpointGET = 'https://gdb.acg.maine.edu:7201/repositories/Hydrology'

sparqlGET = SPARQLWrapper(endpointGET)
sparqlGET.setHTTPAuth(DIGEST)
sparqlGET.setCredentials('sawgraph-endpoint', 'skailab')
sparqlGET.setMethod(GET)
sparqlGET.setReturnFormat(JSON)

## Query

This query uses federation to access the FIO, S2L13_AdminRegions, and Hydrology repositories. While it does so from the Hydrology repository, any repository would work.

The first block finds facilities that are either landfills or DoD sites (via the specified NAICS industry codes) and the S2 cells they are within. It pulls label (facility name) and industry information if it is available as well as geometries.

The second block retrieves the geometries for the S2 cells.

The third block looks for any surface water flowlines that cross the S2 cells that contain landfills or DoD sites. It finds all flowlines downstream of those initial flowlines such that the sum of the length of all connected flowlines is at most 20 km. If available, the name associated with the flowline is retrieved. Geometries are found. All flowlines of type "Coastline" are excluded from the results.

The query is executed and returned as a dataframe.

In [12]:
%%time
query = """
PREFIX fio: <http://sawgraph.spatialai.org/v1/fio#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX naics: <http://sawgraph.spatialai.org/v1/fio/naics#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nhdplusv2: <http://nhdplusv2.spatialai.org/v1/nhdplusv2#>
PREFIX schema: <https://schema.org/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX us_frs: <http://sawgraph.spatialai.org/v1/us-frs#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
    SERVICE <repository:FIO> {
        SELECT * WHERE {
            ?fac rdf:type fio:Facility ;
                 fio:ofIndustry ?code ;
                 kwg-ont:sfWithin ?fac_s2  ;
                 geo:hasGeometry/geo:asWKT ?fac_wkt .
            OPTIONAL { ?fac rdfs:label ?faclabel . }
            OPTIONAL { ?code rdfs:label ?ind . }
            VALUES ?code { naics:NAICS-IndustryCode-562212 naics:NAICS-IndustryCode-928110 naics:NAICS-IndustryCode-92811 }
        }
    }

    SERVICE <repository:Spatial> {
        SELECT * WHERE {
            ?fac_s2 rdf:type kwg-ont:S2Cell_Level13 ;
                    geo:hasGeometry/geo:asWKT ?fac_s2_wkt ;
                    kwg-ont:sfWithin kwgr:administrativeRegion.USA.23 .
        }
    }

    SERVICE <repository:Hydrology> {
        SELECT * WHERE {
            ?fl_us rdf:type hyf:HY_FlowPath ;
            	   kwg-ont:sfCrosses ?fac_s2 ;
            	   hyf:downstreamWaterBodyTC ?fl_ds ;
                   nhdplusv2:hasFTYPE ?fl_us_ftype ;
                   nhdplusv2:hasFlowPathLength ?fl_us_fpl ;
            	   geo:hasGeometry/geo:asWKT ?fl_us_wkt .
            ?fl_us_fpl qudt:quantityValue ?fl_us_fpl_qv .
            ?fl_us_fpl_qv qudt:numericValue ?fl_us_length .
            ?fl_ds rdf:type hyf:HY_FlowPath ;
                   nhdplusv2:hasFTYPE ?fl_ds_ftype ;
                   nhdplusv2:hasFlowPathLength ?fl_ds_fpl ;
                   geo:hasGeometry/geo:asWKT ?fl_ds_wkt .
            ?fl_ds_fpl qudt:quantityValue ?fl_ds_fpl_qv .
            ?fl_ds_fpl_qv qudt:numericValue ?fl_ds_length .
            OPTIONAL { ?fl_us schema:name ?fl_us_name . }
            OPTIONAL { ?fl_ds schema:name ?fl_ds_name . }
            {
                SELECT ?fl_us ?fl_ds (SUM(?fl_length) AS ?path_length) (COUNT(?fl_length) AS ?path_segments) WHERE {
                    ?fl_us rdf:type hyf:HY_FlowPath ;
                    	   kwg-ont:sfCrosses ?fac_s2 ;
                    	   hyf:downstreamWaterBodyTC ?fl .
                    ?fl rdf:type hyf:HY_FlowPath ;
                    	hyf:downstreamWaterBodyTC ?fl_ds ;
                    	nhdplusv2:hasFTYPE ?fl_ftype ;
                    	nhdplusv2:hasFlowPathLength ?fl_fpl .
                    ?fl_fpl qudt:quantityValue ?fl_fpl_qv .
                    ?fl_fpl_qv qudt:numericValue ?fl_length .
                    ?fl_ds nhdplusv2:hasFTYPE ?fl_ds_ftype .
                } GROUP BY ?fl_us ?fl_ds
            }
            FILTER (?path_length < "20.0"^^xsd:float)
        }
    }
}
"""
df = get_sparql_dataframe(endpointGET, query)
print(df.shape)
print(df.head())

(446, 23)
                                                 fac                                               code                                             fac_s2                     fac_wkt                                faclabel  \
0  http://sawgraph.spatialai.org/v1/us-frs-data#d...  http://sawgraph.spatialai.org/v1/fio/naics#NAI...  http://stko-kwg.geog.ucsb.edu/lod/resource/s2....  POINT (-70.26051 44.06723)  MID MAINE WASTE - TO - ENERGY FACILITY   
1  http://sawgraph.spatialai.org/v1/us-frs-data#d...  http://sawgraph.spatialai.org/v1/fio/naics#NAI...  http://stko-kwg.geog.ucsb.edu/lod/resource/s2....  POINT (-70.26051 44.06723)  MID MAINE WASTE - TO - ENERGY FACILITY   
2  http://sawgraph.spatialai.org/v1/us-frs-data#d...  http://sawgraph.spatialai.org/v1/fio/naics#NAI...  http://stko-kwg.geog.ucsb.edu/lod/resource/s2....  POINT (-70.26051 44.06723)  MID MAINE WASTE - TO - ENERGY FACILITY   
3  http://sawgraph.spatialai.org/v1/us-frs-data#d...  http://sawgraph.spatialai.org/v1

# Visualizing on a map

## Partioning the Query Results

The data is split into four separate themed dataframes, one for the facilities, one for the facility S2 cells, one for the flowlines that cross each facility's S2 cells, and one for the downstream flowlines.

The columns with WKT coordinates in each dataframe are converted to Shapely geometry objects prior to converting the dataframes to GeoDataFrames.

In [13]:
df_fac = df[['fac', 'faclabel', 'ind', 'fac_wkt']].copy()
df_fac.drop_duplicates(inplace=True)
df_fac['fac_wkt'] = df_fac['fac_wkt'].apply(wkt.loads)

df_s2 = df[['fac', 'fac_s2', 'faclabel', 'fac_s2_wkt']].copy()
df_s2.drop_duplicates(inplace=True)
df_s2['fac_s2_wkt'] = df_s2['fac_s2_wkt'].apply(wkt.loads)

df_fl_us = df[['fl_us', 'fl_us_length', 'fl_us_name', 'fl_us_wkt']].copy()
df_fl_us.drop_duplicates(inplace=True)
df_fl_us['fl_us_wkt'] = df_fl_us['fl_us_wkt'].apply(wkt.loads)

df_fl_ds = df[['fl_us', 'fl_ds', 'fl_ds_length', 'path_segments', 'path_length', 'fl_ds_name', 'fl_ds_wkt']].copy()
df_fl_ds.drop_duplicates(inplace=True)
df_fl_ds['fl_ds_wkt'] = df_fl_ds['fl_ds_wkt'].apply(wkt.loads)

## Create GeoPandas dataframes for mapping

Convert the above dataframes to GeoDataFrames, setting the WKT columns as the geometry columns and setting the CRS to WGS 84.

In [14]:
%%capture
gdf_fac = gpd.GeoDataFrame(df_fac, geometry='fac_wkt')
gdf_fac.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_s2 = gpd.GeoDataFrame(df_s2, geometry='fac_s2_wkt')
gdf_s2.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_fl_us = gpd.GeoDataFrame(df_fl_us, geometry='fl_us_wkt')
gdf_fl_us.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_fl_ds = gpd.GeoDataFrame(df_fl_ds, geometry='fl_ds_wkt')
gdf_fl_ds.set_crs(epsg=4326, inplace=True, allow_override=True)

## Create Map

Each GeoDataFrame is a layer in the final map.

In [15]:
%%capture
fac_color = 'darkred'
s2_color = 'red'
fl_us_color = 'navy'
fl_ds_color = 'blue'
boundweight = 5

map = gdf_fac.explore(color=fac_color,
                      style_kwds=dict(weight=boundweight),
                      tooltip=True,
                      name='<span style="color: darkred;">Facilities</span>',
                      show=True)
gdf_s2.explore(m=map,
               color=s2_color,
               style_kwds=dict(weight=boundweight),
               tooltip=True,
               name='<span style="color: red;">Facility S2 Cells</span>',
               show=True)
gdf_fl_us.explore(m=map,
                  color=fl_us_color,
                  style_kwds=dict(weight=boundweight),
                  tooltip=True,
                  highlight=False,
                  name='<span style="color: navy;">Crossing Flowlines</span>',
                  show=True)
gdf_fl_ds.explore(m=map,
                  color=fl_ds_color,
                  style_kwds=dict(weight=boundweight),
                  tooltip=True,
                  highlight=False,
                  name='<span style="color: blue;">Downstream Flowlines</span>',
                  show=True)

# folium.TileLayer("stamenterrain", show=False).add_to(map)
# folium.TileLayer("MapQuest Open Aerial", show=False).add_to(map)
folium.LayerControl(collapsed=False).add_to(map)

## Show Map

The map is created inside a Figure box to control its size.

In [16]:
map.save('SAWGraph_UC1_CQ2c_map.html')

fig = Figure(width=800, height=600)
fig.add_child(map)

# Looking Further

Once the results of the primary query are available, it is possible to query for additional information related to specific results.

Here you can view some basic information for flowlines. Select a flowline from the drop down and then press the 'play' button on the left. At their outlet, flowlines can split into multiple different downstream flowlines. This data comes from the National Hydrography Database Plus Version 2 (NHDPlus v2).

In [ ]:
flowline = 'comid/5205294' #@param ['comid/1736840', 'comid/3322162', 'comid/5205294', 'comid/6712619']
query = """
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX saw_water: <http://sawgraph.spatialai.org/v1/saw_water#>
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>

SELECT * WHERE {
    SERVICE <repository:Hydrology> {
        SELECT * WHERE {
            ?fl rdf:type hyf:HY_FlowPath ;
                wdp:P2043 ?fl_length ;
                saw_water:hasCOMID ?fl_comid ;
                saw_water:hasFTYPE ?fl_type ;
                saw_water:hasReachCode ?fl_reachcode ;
                hyf:downstreamWaterbody ?ds_wb .
            OPTIONAL{ ?fl schema:name ?fl_name }
            FILTER(?fl = <https://geoconnex.us/nhdplusv2/""" + flowline + """>)
            FILTER(?ds_wb != <https://geoconnex.us/nhdplusv2/""" + flowline + """>)
        }
    }
}
"""
df2 = get_sparql_dataframe(endpointGET, query).fillna('--')
df2 = df2.iloc[:,[0, 6, 1, 2, 3, 4, 5]]
print(tabulate(df2, headers=['Flowline instance', 'Flowline name', 'Length (km)', 'COMID', 'Type', 'Reach code', 'Next flowline(s) downstream'], tablefmt='rounded_outline', showindex=False))

Facility information comes from the Facility Registry Service (FRS) from the Environmental Protection Agency (EPA). Select a facility from the drop down, press the 'play' button on the left, and you will get some basic information for each of the available facilities. A single facility can be classified in more than one industry.

In [ ]:
facility = 'FRS-Facility.110001830733' #@param ['FRS-Facility.110001830733', 'FRS-Facility.110008437290', 'FRS-Facility.110055618577', 'FRS-Facility.110067529142']
query = """
PREFIX fio: <http://sawgraph.spatialai.org/v1/fio#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX us_frs: <http://sawgraph.spatialai.org/v1/us-frs#>

SELECT ?fac ?fac_name ?admin_region_name ?primary_industry ?fac_frs_id WHERE {
    SERVICE <repository:FIO> {
        SELECT * WHERE {
            ?fac rdf:type fio:Facility ;
                 rdfs:label ?fac_name ;
                 us_frs:primaryIndustry/rdfs:label ?primary_industry ;
                 us_frs:hasFRSId ?fac_frs_id ;
                 kwg-ont:sfWithin ?admin_region .
            FILTER(?fac = <http://sawgraph.spatialai.org/v1/us-frs-data#d.""" + facility + """>)
        }
    }

    SERVICE <repository:S2L13_AdminRegions> {
        SELECT * WHERE {
            ?admin_region rdf:type kwg-ont:AdministrativeRegion_3 .
            ?admin_region rdfs:label ?admin_region_name .
        }
    }
}
"""
df3 = get_sparql_dataframe(endpointGET, query).fillna('--')
print(tabulate(df3, headers=['Facility instance', 'Facility Name', 'Facility location', 'Primary Industry', 'FRS ID'], tablefmt='fancy_outline', showindex=False))